In [1]:
import pandas as pd
import duckdb

In [2]:
con = duckdb.connect()

In [3]:
fitbit_user_day_clean = pd.read_csv("data/clean/fitbit_user_day_clean.csv")

In [4]:
apple_user_day_clean = pd.read_csv("data/clean/apple_user_day_clean.csv")

In [5]:
whoop_user_day_clean = pd.read_csv("data/clean/whoop_user_day_clean.csv")

In [6]:
con.register("fitbit_user_day_clean", fitbit_user_day_clean)
con.register("apple_user_day_clean", apple_user_day_clean)
con.register("whoop_user_day_clean", whoop_user_day_clean)

In [7]:
con.execute("DESCRIBE fitbit_user_day_clean").df()

,column_name,column_type,null,key,default,extra
0,TotalSteps,DOUBLE,YES,None,None,None
1,TrackerDistance,DOUBLE,YES,None,None,None
2,VeryActiveMinutes,DOUBLE,YES,None,None,None
3,FairlyActiveMinutes,DOUBLE,YES,None,None,None
4,LightlyActiveMinutes,DOUBLE,YES,None,None,None
5,SedentaryMinutes,DOUBLE,YES,None,None,None
6,Calories,DOUBLE,YES,None,None,None
7,source,VARCHAR,YES,None,None,None
8,steps_clean,DOUBLE,YES,None,None,None
9,steps_flag_outlier,BIGINT,YES,None,None,None


In [8]:
con.execute("DESCRIBE apple_user_day_clean").df()

,column_name,column_type,null,key,default,extra
0,user_id,VARCHAR,YES,None,None,None
1,date,VARCHAR,YES,None,None,None
2,apple_steps,DOUBLE,YES,None,None,None
3,apple_active_energy,DOUBLE,YES,None,None,None
4,apple_resting_hr,DOUBLE,YES,None,None,None
5,apple_sleep_minutes,DOUBLE,YES,None,None,None


In [9]:
con.execute("DESCRIBE whoop_user_day_clean").df()

,column_name,column_type,null,key,default,extra
0,user_id,VARCHAR,YES,None,None,None
1,date,VARCHAR,YES,None,None,None
2,whoop_recovery,DOUBLE,YES,None,None,None
3,whoop_strain,DOUBLE,YES,None,None,None
4,whoop_hrv,DOUBLE,YES,None,None,None
5,whoop_sleep_score,DOUBLE,YES,None,None,None
6,notes,VARCHAR,YES,None,None,None
7,strain_flag_invalid,BIGINT,YES,None,None,None
8,recovery_flag_invalid,BIGINT,YES,None,None,None
9,sleep_flag_invalid,BIGINT,YES,None,None,None


In [10]:
con.execute("""SELECT
  (SELECT COUNT(*) FROM fitbit_user_day_clean) AS fitbit_rows,
  (SELECT COUNT(*) FROM apple_user_day_clean) AS apple_rows,
  (SELECT COUNT(*) FROM whoop_user_day_clean) AS whoop_rows;""").df()

,fitbit_rows,apple_rows,whoop_rows
0,14299,16896,10620


In [11]:
con.execute("""CREATE OR REPLACE VIEW user_day_spine AS 
SELECT user_id, date FROM fitbit_user_day_clean
UNION
SELECT user_id, date FROM apple_user_day_clean
UNION
SELECT user_id, date FROM whoop_user_day_clean""")

In [12]:
con.execute("SELECT COUNT(*) FROM user_day_spine").df()

,count_star()
0,35822


In [13]:
con.execute("""SELECT user_id, date, COUNT(*)
FROM user_day_spine
GROUP BY user_id, date
HAVING COUNT(*) >1""").df()

,user_id,date,count_star()


In [14]:
con.execute("""
  SELECT COUNT(*) AS dup_keys
  FROM (
    SELECT user_id, date, COUNT(*) AS n
    FROM fitbit_user_day_clean
    GROUP BY user_id, date
    HAVING COUNT(*) > 1
  )
""").df()

,dup_keys
0,0


In [15]:
con.execute("""
  SELECT COUNT(*) AS dup_keys
  FROM (
    SELECT user_id, date, COUNT(*) AS n
    FROM apple_user_day_clean
    GROUP BY user_id, date
    HAVING COUNT(*) > 1
  )
""").df()

,dup_keys
0,0


In [16]:
con.execute("""
  SELECT COUNT(*) AS dup_keys
  FROM (
    SELECT user_id, date, COUNT(*) AS n
    FROM whoop_user_day_clean
    GROUP BY user_id, date
    HAVING COUNT(*) > 1
  )
""").df()

,dup_keys
0,797


In [17]:
con.execute("""
  CREATE OR REPLACE VIEW whoop_user_day_dedup AS
  SELECT
    user_id,
    date,
    AVG(whoop_recovery) AS whoop_recovery,
    AVG(whoop_strain) AS whoop_strain,
    AVG(whoop_hrv) AS whoop_hrv,
    AVG(whoop_sleep_score) AS whoop_sleep_score,
    MIN(notes) AS notes,
    MAX(strain_flag_invalid) AS strain_flag_invalid,
    MAX(recovery_flag_invalid) AS recovery_flag_invalid,
    MAX(sleep_flag_invalid) AS sleep_flag_invalid,
    MAX(hrv_flag_invalid) AS hrv_flag_invalid
  FROM whoop_user_day_clean
  GROUP BY user_id, date
""")

In [18]:
con.execute("""
  SELECT COUNT(*) AS dup_keys
  FROM (
    SELECT user_id, date, COUNT(*) AS n
    FROM whoop_user_day_dedup
    GROUP BY user_id, date
    HAVING COUNT(*) > 1
  )
""").df()

,dup_keys
0,0


In [19]:
con.execute("""CREATE OR REPLACE VIEW user_day_all_vendors AS
SELECT p.*,
f.*,
a.*,
w.*
FROM user_day_spine p
LEFT JOIN fitbit_user_day_clean f
ON p.user_id = f.user_id AND p.date = f.date
LEFT JOIN apple_user_day_clean a
ON p.user_id = a.user_id AND p.date = a.date
LEFT JOIN whoop_user_day_dedup w
ON p.user_id = w.user_id AND p.date = w.date""")

In [20]:
con.execute("""SELECT COUNT(*) AS rows
FROM user_day_all_vendors""").df()

,rows
0,35822


In [21]:
con.execute("""COPY user_day_all_vendors
TO 'data/clean/user_day_all_vendors.csv'
(HEADER, DELIMITER ',')""")